In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
	accuracy_score,
	precision_score,
	recall_score,
	f1_score,
	confusion_matrix,
	roc_curve,
	auc,
	classification_report
)
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [ ]:
def carregar_dados_csv(file):

	try:
		
		df_completo = pd.read_csv(file)

		return df_completo

	except FileNotFoundError:

		print(f"Erro: O arquivo '{file}' não foi encontrado.")
		print("Rode o ZeaMays para gerar o arquivo CSV.")
		return None

	except KeyError as e:

		print(f"Erro: Uma ou mais colunas não encontradas no arquivo: {e}")
		return None

	except Exception as e:

		print(f"Ocorreu um erro inesperado: {e}")
		return None

In [ ]:
metricas = "df_metricas_seq.csv"

df_metricas = carregar_dados_csv(metricas)

if df_metricas is not None:
	print("\nHead do DataFrame:")
	print(df_metricas.shape)
	print(df_metricas.head())

In [ ]:
remover_colunas = ["Chr", "SourceAnnotation", "Start", "End", "Score", "Strand", "Phase", "Attributes", "label"]

df_modificado = df_metricas.drop(columns=remover_colunas)

print("\nDataFrame após remover colunas:")
print(df_modificado.shape)
print(df_modificado.head())

In [ ]:
coluna_alvo = 'COS'

In [ ]:
def verfica_unique (df, coluna):
	
	if coluna in df.columns:

		contagem_valores = df[coluna].value_counts()

		print(f"Contagem de cada valor único na coluna '{coluna}':")
		print(contagem_valores)

		valores_unicos = df[coluna].unique()
		print(f"\nValores únicos na coluna '{coluna}':")
		print(valores_unicos)

		numero_de_valores_unicos = df[coluna].nunique()
		print(f"\nA coluna '{coluna}' possui {numero_de_valores_unicos} valores únicos distintos.")

	else:
		print(f"A coluna '{coluna}' não foi encontrada no DataFrame.")
		print("Colunas disponíveis:", df.columns.tolist())

verfica_unique(df_modificado, coluna_alvo)

In [ ]:
def extrair_elemento_transponivel(texto):
  
	try:
		return texto.split('/')[1]
	except IndexError:
		return
	
df_modificado[coluna_alvo] = df_modificado[coluna_alvo].apply(extrair_elemento_transponivel)

verfica_unique(df_modificado, coluna_alvo)

In [ ]:
ocorrencia = 1

contagens = df_modificado[coluna_alvo].value_counts()

valores_para_manter = contagens[contagens > ocorrencia].index

df_filtrado = df_modificado[df_modificado[coluna_alvo].isin(valores_para_manter)]

verfica_unique(df_filtrado, coluna_alvo)

In [ ]:
df_filtrado.shape

In [ ]:
"""

# Encontra todas as colunas que começam com o prefixo "chaos_"
colunas_para_remover = [col for col in df_filtrado.columns if col.startswith('ZMays/Agrup\chaos_')]

# Cria um novo DataFrame sem essas colunas
df_filtrado = df_filtrado.drop(columns=colunas_para_remover)

print("Colunas removidas:", colunas_para_remover)
print("\nNovo formato do DataFrame:", df_filtrado.shape)
print("Primeiras linhas do DataFrame limpo:")
print(df_filtrado.head())

"""

In [ ]:
df_filtrado.shape

In [ ]:
def random_forest(X_train, y_train, feature_names, caminho_salvar):
    
    print("--- Treinando o Modelo Random Forest ---")
    modelo_rf = RandomForestClassifier(n_estimators=120, class_weight='balanced', random_state=42, n_jobs=-1)
    modelo_rf.fit(X_train, y_train)
    
    print(f"Salvando o modelo em: {caminho_salvar}")
    joblib.dump(modelo_rf, caminho_salvar)
    print("Modelo Random Forest treinado e salvo com sucesso!")

def random_forest_optuna(X_train, y_train, X_test, y_test, feature_names, caminho_salvar, n_trials=100):
    
    print("--- Otimizando e Treinando o Modelo Random Forest com Optuna ---")
    
    def objetivo(trial):
        
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'max_depth': trial.suggest_int('max_depth', 4, 50),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 32),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 32),
            'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
            'random_state': 42,
            'n_jobs': -1
        }
        
        modelo_rf = RandomForestClassifier(**params)
        modelo_rf.fit(X_train, y_train)
        y_pred = modelo_rf.predict(X_test)
        return accuracy_score(y_test, y_pred)

    estudo = optuna.create_study(direction='maximize')
    estudo.optimize(objetivo, n_trials=n_trials)
    
    melhores_params = estudo.best_params
    print(f"Melhores hiperparâmetros encontrados: {melhores_params}")
    
    modelo_rf_optuna = RandomForestClassifier(**melhores_params, random_state=42, n_jobs=-1)
    modelo_rf_optuna.fit(X_train, y_train)
    
    print(f"Salvando o modelo em: {caminho_salvar}")
    joblib.dump(modelo_rf_optuna, caminho_salvar)
    print("Modelo Random Forest (Optuna) treinado e salvo com sucesso!")

def xgboost(X_train, y_train, feature_names, caminho_salvar):
    
    print("--- Treinando o Modelo XGBoost ---")
    num_classes = len(np.unique(y_train))
    params_xgb = {
        'n_estimators': 120,
        'learning_rate': 0.1,
        'max_depth': 4,
        'random_state': 42,
        'use_label_encoder': False,
    }
    if num_classes > 2:
        params_xgb['objective'] = 'multi:softprob'
        params_xgb['eval_metric'] = 'mlogloss'
    else:
        params_xgb['objective'] = 'binary:logistic'
        params_xgb['eval_metric'] = 'logloss'
        
    modelo_xgb = XGBClassifier(**params_xgb)
    modelo_xgb.fit(X_train, y_train)
    
    print(f"Salvando o modelo em: {caminho_salvar}")
    joblib.dump(modelo_xgb, caminho_salvar)
    print("Modelo XGBoost treinado e salvo com sucesso!")

['Helitron' 'LINE' 'LTR' 'MITE' 'SINE' 'TIR']
[0 1 2 3 4 5]


In [ ]:
nome_coluna_alvo = 'COS'

df_modelo = df_filtrado.copy()

# Amostragem de 10% do DataFrame filtrado para um pseudo-treinamento
# df_modelo = df_filtrado.sample(frac=0.02, random_state=42)

if nome_coluna_alvo not in df_modelo.columns:

	print(f"ERRO: Coluna alvo '{nome_coluna_alvo}' não encontrada no DataFrame!")
	
else:

	y_original = df_modelo[nome_coluna_alvo]
	X_original = df_modelo.drop(columns=[nome_coluna_alvo])
	
	print(f"\nVariável Alvo Selecionada: '{nome_coluna_alvo}' (Multiclasse)")
	print("-" * 50)

	# Codificar Features Categóricas em X (One-Hot Encoding)
	colunas_categoricas_X = X_original.select_dtypes(include=['object', 'category']).columns
	X_codificado = pd.get_dummies(X_original, columns=colunas_categoricas_X, drop_first=True)
	feature_names = X_codificado.columns.tolist()

	print("\nFeatures (X) após One-Hot Encoding (primeiras linhas):")
	print(X_codificado.head())
	print(f"Número de features após encoding: {X_codificado.shape[1]}")
	print("-" * 50)

	# Codificar Variável Alvo (y) para formato numérico 0..N-1
	le = LabelEncoder()
	y_codificada = le.fit_transform(y_original) # Converte strings/categorias para 0, 1, 2...

	class_names = le.classes_ # Salva os nomes originais das classes
	
	print(f"\nCodificação da Variável Alvo '{nome_coluna_alvo}':")
	print("Classes originais:", le.classes_)
	print("Classes codificadas (únicas):", np.unique(y_codificada))
	print("Alvo (y) após codificação (primeiras ocorrências):", y_codificada[:5])
	print("-" * 50)

	# Dividir em Dados de Treino e Teste
	try:

		X_train, X_test, y_train, y_test = train_test_split(X_codificado, y_codificada, test_size=0.25, random_state=42, stratify=y_codificada)

	except ValueError:

		print("Aviso: Não foi possível usar 'stratify'. Tentando sem.")

		X_train, X_test, y_train, y_test = train_test_split(X_codificado, y_codificada, test_size=0.25, random_state=42)

	print(f"\nDimensões dos conjuntos de treino/teste estabelecidas.")
	print("-" * 50)

In [ ]:
# Caminhos para salvar os modelos
caminho_rf = "models/modelo_rf.joblib"
caminho_rf_optuna = "models/modelo_rf_optuna.joblib"
caminho_xgb = "models/modelo_xgb.joblib"

# Treinar e salvar cada modelo
random_forest(X_train, y_train, feature_names, caminho_rf)
random_forest_optuna(X_train, y_train, X_test, y_test, feature_names, caminho_rf_optuna)
xgboost(X_train, y_train, feature_names, caminho_xgb)

In [ ]:
def carregar_e_plotar_modelos(caminhos_modelos, X_test, y_test, class_names, feature_names):
  
    for nome_modelo, caminho in caminhos_modelos.items():
        print(f"--- Carregando e Avaliando: {nome_modelo} ---")
        
        try:
            modelo_carregado = joblib.load(caminho)
        except FileNotFoundError:
            print(f"Arquivo do modelo não encontrado em: {caminho}")
            continue

        y_pred = modelo_carregado.predict(X_test)

        # Relatório de Classificação
        print("\nRelatório de Classificação:")
        print(classification_report(y_test, y_pred, target_names=class_names))

        # Matriz de Confusão
        plt.style.use('seaborn-v0_8-whitegrid')
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=class_names, yticklabels=class_names)
        plt.title(f'Matriz de Confusão - {nome_modelo}')
        plt.ylabel('Verdadeiro')
        plt.xlabel('Previsto')
        plt.tight_layout()
        plt.show()

        # Importância das Features
        if hasattr(modelo_carregado, 'feature_importances_'):
            importances = modelo_carregado.feature_importances_
            indices = np.argsort(importances)[::-1]
            num_features_to_plot = min(len(feature_names), 20)
            
            plt.figure(figsize=(12, max(6, num_features_to_plot * 0.3)))
            plt.title(f"Importância das Features - {nome_modelo} (Top {num_features_to_plot})")
            sns.barplot(x=importances[indices][:num_features_to_plot],
                        y=[feature_names[i] for i in indices[:num_features_to_plot]])
            plt.xlabel("Importância Relativa")
            plt.ylabel("Feature")
            plt.tight_layout()
            plt.show()

# Dicionário com os nomes dos modelos e os caminhos onde foram salvos
caminhos_dos_modelos = {
    "Random Forest": "models/modelo_rf.joblib",
    "Random Forest (Optuna)": "models/modelo_rf_optuna.joblib",
    "XGBoost": "models/modelo_xgb.joblib"
}

# Carregar os modelos e gerar os gráficos
carregar_e_plotar_modelos(caminhos_dos_modelos, X_test, y_test, class_names, feature_names)